# Esolang Interpreters #4 - Boolfuck Interpreter

## The Task
Write a Boolfuck interpreter which accepts up to two arguments. The first (required) argument is the Boolfuck code in the form of a string. The second (optional) argument is the input passed in by the end-user (i.e. as actual characters not bits) which should default to "" if not provided. Your interpreter should return the output as actual characters (not bits!) as a string.

```js
function boolfuck (code, input = "")
```

Preloaded for you is a function `brainfuckToBoolfuck()`/`brainfuck_to_boolfuck()`/`BrainfuckToBoolfuck()` which accepts 1 required argument (the Brainfuck code) and returns its Boolfuck equivalent should you find it useful.

Please note that your interpreter should simply ignore any non-command characters. **This will be tested in the test cases.**

## My Solution to the Kata

```js
let boolfuck = (code, input = "") => {
    const inputBits = [...input].flatMap(char =>
        char.charCodeAt().toString(2).padStart(8, '0').split("").reverse().map(Number)
    );

    let memory = [0], pointer = 0, inputPointer = 0, outputBits = "", stack = [], instructionPointer = 0;

    while (instructionPointer < code.length) {
        const cmd = code[instructionPointer];
        switch (cmd) {
            case '>': pointer++; if (pointer >= memory.length) memory.push(0); break;
            case '<': pointer--; if (pointer < 0) { memory.unshift(0); pointer = 0; } break;
            case '+': memory[pointer] ^= 1; break;
            case ';': outputBits += memory[pointer]; break;
            case ',': memory[pointer] = inputPointer < inputBits.length ? inputBits[inputPointer++] : 0; break;
            case '[':
                if (memory[pointer] === 0) {
                    let balance = 1;
                    while (balance > 0 && ++instructionPointer < code.length) {
                        if (code[instructionPointer] === '[') balance++;
                        else if (code[instructionPointer] === ']') balance--;
                    }
                } else stack.push(instructionPointer);
                break;
            case ']':
                if (memory[pointer] !== 0) instructionPointer = stack[stack.length - 1];
                else stack.pop();
                break;
        }
        instructionPointer++;
    }

    const bytes = outputBits.match(/.{1,8}/g) || [];
    return bytes.map(byte =>
        String.fromCharCode(parseInt(byte.padEnd(8, '0').split("").reverse().join(""), 2))
    ).join('');
}
```

## Explanation

### The Function

```js
let boolfuck = (code, input = "") => { ... }
```

- **`code`:** Required input. This represents as a string that the interpreter will execute.

- **`input = ""`:** An optional argument. This defaults the to an empty string (`""`).

### Convert Input Into bits

```js
const inputBits = [...input].flatMap(char =>
        char.charCodeAt().toString(2).padStart(8, '0').split("").reverse().map(Number)
);
```

- The `input` string is converted into an array of characters using `[...]input` <br> ( [[...input] The Spread Operator](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Operators/Spread_syntax) ).

- For each character in the input, its Unicode code point is obtained using `charCodeAt()`.

- This code point is then converted into an 8-bit binary string (with leading zeros if needed), using `toString(2).padStart(8, '0')`.

- The string is reversed `(split("").reverse())` so that the least significant bit (LSB) comes first.

- The reversed bits are then converted into an array of numbers (`map(Number)`).

This process effectively converts the input string into a bit array, where each bit corresponds to a value from the binary representation of the input characters. 

### Initialize Memory and Other Variables

```js
let memory = [0], pointer = 0, inputPointer = 0, outputBits = "", stack = [], instructionPointer = 0;
```

- **`memory`:** A memory array initialized with a single `0` value, representing the current memory cell. The memory pointer (`pointer`) will refer to the current cell.

- **`pointer`:** Points to the current memory cell, initialized to `0`.

- **`inputPointer`:** Tracks the position of the current input bit, starting at `0`.

- **`outputBits`:** An empty string to store the output as binary bits.

- **`stack`:** A stack to manage loop operations (for handling the `[` and `]`).

- **`instructionPointer`:** Tracks the current instruction's position within the code.

### Interpreter Loop

```js
while (instructionPointer < code.length) {
    const cmd = code[instructionPointer];
    switch (cmd) {
        case '>': pointer++; if (pointer >= memory.length) memory.push(0); break;
        case '<': pointer--; if (pointer < 0) { memory.unshift(0); pointer = 0; } break;
        case '+': memory[pointer] ^= 1; break;
        case ';': outputBits += memory[pointer]; break;
        case ',': memory[pointer] = inputPointer < inputBits.length ? inputBits[inputPointer++] : 0; break;
        case '[':
            if (memory[pointer] === 0) {
                let balance = 1;
                while (balance > 0 && ++instructionPointer < code.length) {
                    if (code[instructionPointer] === '[') balance++;
                    else if (code[instructionPointer] === ']') balance--;
                }
            } else stack.push(instructionPointer);
            break;
        case ']':
            if (memory[pointer] !== 0) instructionPointer = stack[stack.length - 1];
            else stack.pop();
            break;
    }
    instructionPointer++;
}
```

1. **`while (instructionPointer < code.length)`:** The loop runs through each instruction in the `code` string.

2. **`switch(cmd)`:** Checks the current instruction `cmd` form the `code` string and performs corresponding actions:
    - **`>`:** Increment the memory pointer (`pointer`). If the pointer exceeds the current memory array size, a new memory cell (`0`) is added.
    - **`<`:** Decrement the memory pointer. If it goes below `0`, a new memory cell (`0`) is added at the beginning, and the pointer is reset to `0`.
    - **`+`:** Flip the current bit at the memory pointer. This operation uses the XOR (`^= 1`) to toggle the bit between `0` and `1`.
    - **`;`:** Append the current bit from the memory cell to the outputBits string.
    - **`,`:** Set the current memory cell to a bit from the input (from `inputBits`). The `inputPointer` tracks the current position in the input, and if all input bits are used, the memory cell is set to `0`.
    - **`[`:** This command starts a loop. If the current memory cell is `0`, the interpreter jumps to the matching `]` (skipping the loop). Otherwise, the current instruction pointer is pushed onto the stack.
    - **`]`:** If the current memory cell is non-zero, jump back to the matching `[` (loop continuation). If the memory cell is `0`, pop the instruction from the stack and continue.

### Convert The Output Binary String to Actual Characters

```js
const bytes = outputBits.match(/.{1,8}/g) || [];
return bytes.map(byte =>
    String.fromCharCode(parseInt(byte.padEnd(8, '0').split("").reverse().join(""), 2))
).join('');
```

- The `outputBits` string is grouped into chunks of 8 bits using `match(/.{1,8}/g)`, which correspond to bytes.

- For each byte:
    - It is padded with zeros to ensure it is 8 bits (`padEnd(8, '0')`). <br> [`padEnd()`](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/String/padEnd)
    - The byte is reversed to restore the original order of bits.
    - The byte is then converted back to a decimal number using `parseInt(..., 2)`. <br> [`parseInt()`](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/parseInt)
    - `string.fromCharCode()` converts the number back to a character. <br> [`fromCharCode()`](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/String/fromCharCode)

- The resulting characters are joined together into a final string.

## On a Final Note

The **Boolfuck interpreter** executes the code by emulating its fundamental memory architecture and instruction set. It manipulates memory, manages input and output in binary format, and facilitates looping. It provides the output as a string of characters instead of raw bits.